### Image-derived Mesh Generation with fTetWild 

In this notebook, we demonstrate how to generate a multidomain brain mesh using boolean operations on surface meshes derived from MRI imaging data with [fTetWild](https://wildmeshing.github.io/ftetwild/).

Even though the resulting mesh is slightly different, the surfaces used are the same as used in 
Causemann, M., Vinje, V. & Rognes, M.E. Human intracranial pulsatility during the cardiac cycle: a computational modelling framework. Fluids Barriers CNS 19, 84 (2022). https://doi.org/10.1186/s12987-022-00376-2.
Due to their reduced memory footprint we use binary .ply files here, but it would equally be possible to use .stl here.

In [ ]:
import wildmeshing as wm
import json
import numpy as np
import pyvista as pv
import meshio
porous_id = 1
fluid_id = 2

tetra = wm.Tetrahedralizer(epsilon=0.001,edge_length_r=0.03,
                           coarsen=False)
tetra.load_csg_tree(json.dumps(
    {"operation":"union",
     "right":
          {
          "operation":"union",
          "left":{"operation":"union",
                  "left":
                      {"operation":"union", "left": "real_brain_surfaces/median_aperture.ply",
                                            "right": "real_brain_surfaces/fourth_ventricle.ply"},
                  "right":
                      {"operation":"union", "left": "real_brain_surfaces/foramina.ply",
                                            "right": "real_brain_surfaces/third_ventricle.ply"},
                 },
          "right":{"operation":"union", "left": "real_brain_surfaces/aqueduct.ply",
                          "right":"real_brain_surfaces/lateral_ventricles.ply"},
          },
     "left":
         {"operation":"union", "left": "real_brain_surfaces/csf.ply",
                              "right": "real_brain_surfaces/parenchyma.ply"},
    }
))
tetra.tetrahedralize()
points, cells, marker = tetra.get_tet_mesh()

In [ ]:
mesh = meshio.Mesh(points, [("tetra", cells)],
                   cell_data={"subdomains": [marker.ravel()]})
mesh.write("mesh.xdmf")

In [ ]:
mesh = pv.read("mesh.xdmf").clean()
mesh["subdomains"][mesh["subdomains"]==1] = 9
ventricles = mesh.extract_cells(np.isin(marker, [3,4,5,6,7,8]))
pl = pv.Plotter()
pl.add_mesh(mesh.clip(normal=(1,0,0),crinkle=True), clim=(1,9),
            show_edges=True, show_scalar_bar=False)
pl.add_mesh(ventricles, clim=(1,9), show_edges=True,
            show_scalar_bar=False)
pl.background_color = "white"
pl.view_yz()
pl.show()
_= pl.screenshot("brain_mesh.png", scale=5)

In [ ]:
pl = pv.Plotter()
pl.add_mesh(ventricles, clim=(1,9), show_edges=True,
            show_scalar_bar=False)
pl.background_color = "white"
pl.view_vector([0.7,0.5,-0.2])
pl.show()
_= pl.screenshot("ventricles.png", scale=5)